### 일주일치 정산

In [160]:
import pymysql
from datetime import datetime, timedelta

## 날짜 찾아서  변수 추가

now = datetime.now() # 현재
date = now.date() - timedelta(days=6)

# SQL 문에 넣을 쿼리 작성 (전체 찾기)
sqlery = f"SELECT * FROM anchovy_train_train WHERE train_date >= '{date}' " 

# SQL 문 조회
connect = pymysql.connect(host="database-dev.cclpxl5ttlgd.ap-northeast-2.rds.amazonaws.com", user="develop",
               password="anchovy1!", db="anchovy", charset="utf8", autocommit=True, # .autocommit 결과 반영
               cursorclass=pymysql.cursors.DictCursor) #DB 조회시 컬럼명을 동시에 보여준다
cur = connect.cursor() # 연결 (Cusor 생성)
sql = sqlery #Train 테이블 전체를 불러옴 (sql 문 정의)
cur.execute(sql) # Sql문 실행


weekdata = cur.fetchall() # 나열 함수
username = []
for i in weekdata:
    username.append(i['username'])

usernames = list(set(username))
score = {}

for username in usernames:
    for all_data in weekdata:
        if all_data['username'] == username:
            if username in score.keys():
                score[username] += all_data['train_accurate_count']
            else:
                score[username] = all_data['train_accurate_count']
            

connect.close() #Sql 문을 종료

{'zza': 82, 'psj': 153, 'Qotrl': 47, 'psj2': 28, 'zzayo': 48}


## lv 비교

In [208]:
character_lv = {1:120, 2:150, 3:180, 4: 210}
# SQL 문에 넣을 쿼리 작성 (전체 찾기)
coupon = "SELECT * FROM anchovy_common_user_status WHERE character_lv > 0"  # 이전 쿠폰 값
update_coupon = f"UPDATE anchovy_common_user_status SET coupon=%s WHERE username=%s" # 현재 쿠폰값 업데이트
reset_week = f"UPDATE anchovy_common_user_status SET week_train_count=0"

# SQL 문 조회
connect = pymysql.connect(host="database-dev.cclpxl5ttlgd.ap-northeast-2.rds.amazonaws.com", user="develop",
               password="anchovy1!", db="anchovy", charset="utf8", autocommit=True, # .autocommit 결과 반영
               cursorclass=pymysql.cursors.DictCursor) #DB 조회시 컬럼명을 동시에 보여준다

coupon_connect = connect.cursor() # 연결 (Cusor 생성)
coupon_connect.execute(coupon) # Sql문 실행

user_datas = coupon_connect.fetchall()


for username in score:
    for user_data in user_datas:
        if user_data['username'] == username:
            if score[username] >= character_lv[user_data['character_lv']]:
                new_coupon = user_data['coupon'] + 1
                new_connect = connect.cursor()
                new_connect.execute(update_coupon, (new_coupon,username))

reset_connect = connect.cursor()
reset_connect.execute(reset_week)
                
connect.close() #Sql 문을 종료